<a href="https://colab.research.google.com/github/helloankur/pcb-defect-detect-yolov5-/blob/main/PCB_defect_model_train.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone https://github.com/ultralytics/yolov5  # clone repo

fatal: destination path 'yolov5' already exists and is not an empty directory.


In [2]:
!git clone https://github.com/tangsanli5201/DeepPCB.git

fatal: destination path 'DeepPCB' already exists and is not an empty directory.


In [3]:
import yaml
import glob
import os
import shutil
from tqdm import tqdm
import cv2
import pandas as pd
import numpy as np
import random
import matplotlib.pyplot as plt



In [4]:
os.makedirs('/content/tmp/images/train', exist_ok=True)
os.makedirs('/content/tmp/images/val', exist_ok=True)
os.makedirs('/content/tmp/images/test', exist_ok=True)
os.makedirs('/content/tmp/labels/train', exist_ok=True)
os.makedirs('/content/tmp/labels/val', exist_ok=True)
os.makedirs('/content/tmp/labels/test', exist_ok=True)

In [5]:
# %cd yolov5
%pip install -qr requirements.txt  # install dependencies

import torch
from IPython.display import Image, clear_output  # to display images

clear_output()
print(f"Setup complete. Using torch {torch.__version__} ({torch.cuda.get_device_properties(0).name if torch.cuda.is_available() else 'CPU'})")


Setup complete. Using torch 1.9.0+cu102 (CPU)


In [6]:
folder_img_dir = glob.glob('pcb_defect/DeepPCB-master/**/*.jpg', recursive=True)

train_val = []
test = []

with open('/content/DeepPCB/PCBData/test.txt') as f:
    for line in f.readlines():
        test.append(line)

with open('/content/DeepPCB/PCBData/trainval.txt') as f1:
    for line in f1.readlines():
        train_val.append(line)

val_split = int(len(train_val) * 0.1)
random.seed(101)
random.shuffle(train_val)

train = train_val[val_split:]  # create train dataset from train_val text file
val = train_val[:val_split]  # create val dataset from train_val text file



In [7]:
def to_csv(data):
    columns = list(data.keys())
    values = list(data.values())
    arr_len = len(values)
    df = pd.DataFrame(np.array(values, dtype=object).reshape(1, arr_len), columns=columns).reset_index()
    # print(df)
    return df

In [8]:
df_all = []

In [9]:
datasets = [train, val, test]
dir_nme = ['train', 'val', 'test']
for k in tqdm(datasets):
    folder_num = (datasets.index(k))
    for i in tqdm(k):
        data = {}
        for j in i.split(" "):
            print_buffer = []
            if '.jpg' in j:
                path_jpg = '/content/DeepPCB/PCBData/' + j
                head_img, tail_img = os.path.split(path_jpg)
                # print(head)

                org_name = (tail_img.strip('.jpg'))
                # print(tail+'_test.jpg')

                jpg_path = (head_img + '/' + org_name + '_test.jpg')
                img_read = cv2.imread(jpg_path)
                shutil.copy(jpg_path, '/content/tmp/images/' + str(dir_nme[folder_num]))
                os.rename('/content/tmp/images/' + str(dir_nme[folder_num]) + '/' + org_name + '_test.jpg',
                          '/content/tmp/images/' + str(dir_nme[folder_num]) + '/' + tail_img)

            if '.txt' in j:
                path_txt = '/content/DeepPCB/PCBData/' + j.strip('\n')

                with open(path_txt, 'r') as f:
                    for line in f.readlines():
                        annot = (line.strip("\n").split(" "))

                        x_min = int(annot[0])
                        y_min = int(annot[1])
                        x_max = int(annot[2])
                        y_max = int(annot[3])
                        label = int(annot[4])

                        # Transform the bbox co-ordinates as per the format required by YOLO v5
                        b_center_x = (x_min + x_max) / 2
                        b_center_y = (y_min + y_max) / 2
                        b_width = (x_max - x_min)
                        b_height = (y_max - y_min)

                        # Normalise the co-ordinates by the dimensions of the image
                        image_w, image_h, image_c = img_read.shape
                        b_center_x /= image_w
                        b_center_y /= image_h
                        b_width /= image_w
                        b_height /= image_h

                        print_buffer.append(
                            "{} {:.3f} {:.3f} {:.3f} {:.3f}".format(label, b_center_x, b_center_y, b_width, b_height))

                        head_txt, tail_txt = os.path.split(path_txt)
                        # print(tail_txt)

                        print("\n".join(print_buffer),
                              file=open('/content/tmp/labels/' + dir_nme[folder_num] + '/' + tail_txt.strip('\n'), "w"))

                        id = tail_txt.strip('.txt')

                        data['id'] = id
                        data['label'] = label
                        data['x_min'] = x_min
                        data['y_min'] = y_min
                        data['x_max'] = x_max
                        data['y_max'] = y_max
                        data['b_center_x'] = b_center_x
                        data['b_center_y'] = b_center_y
                        data['b_width'] = b_width
                        data['b_height'] = b_height
                        data['image_w'] = image_w
                        data['image_h'] = image_h
                        data['split'] = dir_nme[folder_num]

                        # Convert Dic data to  dataframe

                        df = to_csv(data)
                        df_all.append(df)



final_df = pd.concat(df_all, ignore_index=True)

del final_df['index']

print(final_df)

final_df.to_csv('meta.csv', index=False)


100%|██████████| 3/3 [00:27<00:00,  9.05s/it]


             id label x_min y_min  ...   b_height image_w image_h  split
0      20085138     1    81   467  ...   0.059375     640     640  train
1      20085138     2    52   564  ...  0.0453125     640     640  train
2      20085138     2     4   568  ...  0.0484375     640     640  train
3      20085138     4   266   486  ...   0.046875     640     640  train
4      20085138     4   410   529  ...  0.0453125     640     640  train
...         ...   ...   ...   ...  ...        ...     ...     ...    ...
10008  12000593     1   271   349  ...    0.06875     640     640   test
10009  12000593     2   182   160  ...   0.071875     640     640   test
10010  12000593     2   330   131  ...  0.0546875     640     640   test
10011  12000593     3   375   180  ...       0.05     640     640   test
10012  12000593     4   329   274  ...    0.04375     640     640   test

[10013 rows x 13 columns]


In [10]:
os.listdir('/content/yolov5/data')

['Objects365.yaml',
 'coco128.yaml',
 'SKU-110K.yaml',
 'VisDrone.yaml',
 'coco.yaml',
 'VOC.yaml',
 'GlobalWheat2020.yaml',
 'Argoverse.yaml',
 'images',
 'hyps',
 'scripts',
 'xView.yaml']

In [12]:
data_yaml=dict(train='/content/tmp/images/train',
               val='/content/tmp/images/val',
               test='/content/tmp/images/test',
               nc=7,
               names=["background","open", "short", "mousebite", "spur", "copper",  "pin-hole"])

In [13]:
with open ("/content/yolov5/data/data.yaml",'w') as outfile:
  yaml.dump(data_yaml,outfile,default_flow_style=True)

In [14]:
with open ("/content/yolov5/data/data.yaml",'r') as f:
  for line in f:
    print(line)

{names: [background, open, short, mousebite, spur, copper, pin-hole], nc: 7, test: /content/tmp/images/test,

  train: /content/tmp/images/train, val: /content/tmp/images/val}



In [15]:
%cd yolov5

/content/yolov5


In [ ]:
!python train.py --img 640 \
                 --batch 4 \
                 --epochs 20 \
                 --data data.yaml \
                 --weights yolov5l.pt \
                 --project PCB_fault_detect 

train: weights=yolov5l.pt, cfg=, data=data.yaml, hyp=data/hyps/hyp.scratch.yaml, epochs=20, batch_size=4, imgsz=640, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, evolve=None, bucket=, cache=None, image_weights=False, device=, multi_scale=False, single_cls=False, adam=False, sync_bn=False, workers=8, project=PCB_fault_detect, entity=None, name=exp, exist_ok=False, quad=False, linear_lr=False, label_smoothing=0.0, upload_dataset=False, bbox_interval=-1, save_period=-1, artifact_alias=latest, local_rank=-1, freeze=0
github: up to date with https://github.com/ultralytics/yolov5 ✅
YOLOv5 🚀 v5.0-380-g11f85e7 torch 1.9.0+cu102 CPU

hyperparameters: lr0=0.01, lrf=0.2, momentum=0.937, weight_decay=0.0005, warmup_epochs=3.0, warmup_momentum=0.8, warmup_bias_lr=0.1, box=0.05, cls=0.5, cls_pw=1.0, obj=1.0, obj_pw=1.0, iou_t=0.2, anchor_t=4.0, fl_gamma=0.0, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, degrees=0.0, translate=0.1, scale=0.5, shear=0.0, perspective=0.0, flipud=0.0, f